# Set-up: Connect to Drive & Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
! ls drive/Shareddrives/A2R\ Lab/Rigid\ Body\ Dynamics/Fall\ 2023/GRiD-Fall-2022-master

In [ ]:
%cd drive/Shareddrives/A2R\ Lab/Rigid\ Body\ Dynamics/Fall\ 2023/GRiD-Fall-2022-master

/content/drive/Shareddrives/A2R Lab/Rigid Body Dynamics/Fall 2023/GRiD-Fall-2022-master


In [2]:
!git clone https://github.com/eden-chung/RNEAPytorch.git

Cloning into 'RNEAPytorch'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 100 (delta 23), reused 86 (delta 12), pack-reused 0
Receiving objects: 100% (100/100), 368.14 KiB | 4.14 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [3]:
%cd RNEAPytorch

/content/RNEAPytorch


In [12]:
!pip install cupy-cuda12x

# Set-up: Install CuPy and Relevant Libraries

In [13]:
import numpy as np
import cupy as cp
from timeit import default_timer as timer

# Set-up: Extract Vectors/Matrices from Robot Object

In [5]:
from URDFParser import URDFParser
from URDFParser import Robot
from util import parseInputs, printUsage, validateRobot, initializeValues, printErr
from RBDReference import RBDReference
from GRiDCodeGenerator import GRiDCodeGenerator

In [6]:
parser = URDFParser()
robot = parser.parse('/content/RNEAPytorch/URDFParser/iiwa.urdf')

validateRobot(robot)

reference = RBDReference(robot)
#n is the number of joints
q, qd, u, n = initializeValues(robot, MATCH_CPP_RANDOM = True)

print("q", q)
print("qd", qd)
print("u", u)
print("n", n)

Link [base] does not have an origin. Assuming this is the fixed world base frame. Else there is an error with your URDF file.
Link [base] does not have inertial properties. Assuming this is the fixed world base frame. Else there is an error with your URDF file.
------------------------------------------
Assumed Input Joint Configuration Ordering
------------------------------------------
iiwa_joint_1
iiwa_joint_2
iiwa_joint_3
iiwa_joint_4
iiwa_joint_5
iiwa_joint_6
iiwa_joint_7
----------------------------
Total of n = 7 joints
----------------------------
q [-0.3369  1.2966 -0.6775 -1.4218 -0.7067 -0.135  -1.1495]
qd [ 0.433  -0.4216 -0.6454 -1.8605 -0.0131 -0.4583  0.7412]
u [ 0.7418  1.9284 -0.9039  0.0334  1.1799 -1.946   0.3287]
n 7


In [7]:
#initialize robot matrices & vectors
parent_id_arr = []
S_arr = []
Imat_arr = []
for ind in range(n):
  parent_id_arr.append(robot.get_parent_id(ind))
  S_arr.append(robot.get_S_by_id(ind).astype(np.float64))
  Imat_arr.append(robot.get_Imat_by_id(ind))

parent_id_arr = np.array(parent_id_arr)
print("parent_id_arr shape:", parent_id_arr.shape)
S_arr = np.array(S_arr)
print("S_arr shape:", S_arr.shape)
Imat_arr = np.array(Imat_arr)
print("Imat_arr shape:", Imat_arr.shape)

parent_id_arr shape: (7,)
S_arr shape: (7, 6)
Imat_arr shape: (7, 6, 6)


In [8]:
#write xmat functions to file
import array
import sys
import os
import inspect

for ind in range(n):
  with open(f'/content/xmat{ind}.py', 'w') as f:
      original_stdout = sys.stdout
      sys.stdout = f
      try:
          print("from numpy import array, sin, cos")
          print()
          content = robot.get_Xmat_Func_by_id(ind)
          source_code = inspect.getsource(content)
          print(source_code)
          # print(content)
      finally:
          sys.stdout = original_stdout
          f.close()

In [9]:
#store xmat functions into xmat array
py_file_location = "/content"
sys.path.append(os.path.abspath(py_file_location))
import xmat0, xmat1, xmat2, xmat3, xmat4, xmat5, xmat6

xmat_func_arr = []
xmat_func_arr.append(xmat0._lambdifygenerated(q[ind]))
xmat_func_arr.append(xmat1._lambdifygenerated(q[ind]))
xmat_func_arr.append(xmat2._lambdifygenerated(q[ind]))
xmat_func_arr.append(xmat3._lambdifygenerated(q[ind]))
xmat_func_arr.append(xmat4._lambdifygenerated(q[ind]))
xmat_func_arr.append(xmat5._lambdifygenerated(q[ind]))
xmat_func_arr.append(xmat6._lambdifygenerated(q[ind]))

xmat_func_arr = np.array(xmat_func_arr)
print("xmat_func_arr shape:", xmat_func_arr.shape)

xmat_func_arr shape: (7, 6, 6)


# Helper functions for RNEA: cross_operator, mxS, vxIv

# **Cross Operator**

Batched Cross Operator function

In [14]:
def cross_operator_batched(d_vec, d_output):
        # for any vector v, computes the operator v x
        # vec x = [wx   0]
        #         [vox wx]
        #(crm in spatial_v2_extended)

      # Compute the first part of the cross operator
      d_output[0, 1, :] = -d_vec[2, :]
      d_output[0, 2, :] = d_vec[1, :]
      d_output[1, 0, :] = d_vec[2, :]
      d_output[1, 2, :] = -d_vec[0, :]
      d_output[2, 0, :] = -d_vec[1, :]
      d_output[2, 1, :] = d_vec[0, :]

      # Compute the second part of the cross operator
      d_output[3, 1, :] = -d_vec[5, :]
      d_output[3, 2, :] = d_vec[4, :]
      d_output[3, 4, :] = -d_vec[2, :]
      d_output[3, 5, :] = d_vec[1, :]
      d_output[4, 0, :] = d_vec[5, :]
      d_output[4, 2, :] = -d_vec[3, :]
      d_output[4, 3, :] = d_vec[2, :]
      d_output[4, 5, :] = -d_vec[0, :]
      d_output[5, 0, :] = -d_vec[4, :]
      d_output[5, 1, :] = d_vec[3, :]
      d_output[5, 3, :] = -d_vec[1, :]
      d_output[5, 4, :] = d_vec[0, :]

In [15]:
# COMPARING to batched
batch_size = 100
h_vec_batched = np.ones((6,batch_size),  dtype=np.float64)
d_vec_batched = cp.asarray(h_vec_batched)
h_output_batched = np.zeros((6, 6, batch_size), dtype=cp.float64)
d_output_batched = cp.zeros((6, 6, batch_size), dtype=cp.float64)

#on CPU
cross_operator_batched(h_vec_batched, h_output_batched) #warm-up once
print("cross operator output shape: ", h_output_batched.shape)
startnext = timer()
for i in range(100):
  cross_operator_batched(h_vec_batched, h_output_batched)
print("CPU Batched No JIT: " + str(timer() - startnext))

#On GPU
cross_operator_batched(d_vec_batched, d_output_batched) #warm-up once
startnext = timer()
for i in range(100):
  cross_operator_batched(d_vec_batched, d_output_batched)
cp.cuda.Stream.null.synchronize()
print("CuPy Batched: " + str((timer() - startnext)))

cross operator output shape:  (6, 6, 100)
CPU Batched No JIT: 0.003129974999978913
CuPy Batched: 0.042202575000032994


# **mxS**

In [18]:
def mxS_numpy(S, vec, vec_output, mxS_output, alpha):
        # returns the spatial cross product between vectors S and vec. vec=[v0, v1 ... vn] and S = [s0, s1, s2, s3, s4, s5]
        # derivative of spatial motion vector = v x m
        #output should be
        #return(alpha * np.dot(self.cross_operator(vec), S)) brian's code
        #return(alpha * np.dot(cross_oper_jit(vec), S)) our jit code
        if alpha is None:
          alpha = 1
        cross_operator_batched(vec, vec_output)
        # mxS_output = alpha * np.tensordot(vec_output[:, :, :], S[:, :], axes=([1], [0]))
        mxS_output = alpha * np.sum(vec_output * S, axis=1)
        # Iterate over each batch index

        #for i in range(batch_size):
        #  dot_product = np.dot(vec_output[:, :, i], S[:, i])
          # Accumulate the result
         # mxS_output[:,i] = dot_product



def mxS_cupy(S, vec, vec_output, mxS_output, alpha):
        # returns the spatial cross product between vectors S and vec. vec=[v0, v1 ... vn] and S = [s0, s1, s2, s3, s4, s5]
        # derivative of spatial motion vector = v x m
        #return(alpha * np.dot(self.cross_operator(vec), S)) brian's code
        #return(alpha * np.dot(cross_oper_jit(vec), S)) our jit code
        #if alpha is None:
        #  alpha = 1
        #cross_operator_batched(vec, vec_output)
        ##mxS_output = alpha * cp.tensordot(vec_output[:, :, :], S[:, :], axes=([1], [0]))
        #mxS_output = alpha * cp.sum(vec_output * S_broadcasted, axis=1)

        return(alpha * np.dot(cross_operator_batched(vec), S))

        #for i in range(batch_size):
        #  dot_product = cp.dot(vec_output[:, :, i], S[:, i])
          # Accumulate the result
         # mxS_output[:,i] = dot_product



**Testing the Batched mxS Method on CPU vs. GPU with CuPy**

In [20]:
# COMPARING to batched
batch_size = 100
# vec is a 6 by 1 matrix
h_vec_batched = np.ones((6,batch_size),  dtype=np.float64)
d_vec_batched = cp.asarray(h_vec_batched)

#S should be a 6 by 1 matrix
h_s_vec_batched = np.ones((6, 1, batch_size),  dtype=np.float64)
d_s_vec_batched = cp.asarray(h_s_vec_batched)

#vec output is a 6 by 6 matrix
h_output_batched = np.zeros((6, 6, batch_size), dtype=np.float64)
d_output_batched = cp.zeros((6, 6, batch_size), dtype=cp.float64)

#mxS output is a 6 by 6 matrix
#TODO: mxS output should be (6, ) not (6, 6), need to change in CPU batched
h_mxS_output_batched = np.zeros((6, batch_size), dtype=np.float64)
d_mxS_output_batched = cp.zeros((6, batch_size), dtype=cp.float64)

alpha = 0.1

#CPU/with np.tensordot
mxS_numpy(h_s_vec_batched, h_vec_batched, h_output_batched, h_mxS_output_batched, alpha) #warm-up once
print("Shape of CPU mxS output: ", h_mxS_output_batched.shape)
startnext = timer()
for i in range(100):
  mxS_numpy(h_s_vec_batched, h_vec_batched, h_output_batched, h_mxS_output_batched, alpha)
print("CPU without jit Batched mxS: " + str(timer() - startnext))


Shape of CPU mxS output:  (6, 100)
CPU without jit Batched mxS: 0.007932444000005034


# vxIv

**Batched vxIv Method**

In [21]:
def vxIv_numpy(vec, Imat, res, batch_size):
        # necessary component in differentiating Iv (product rule).
        # We express I_dot x v as v x (Iv) (see Featherstone 2.14)
        # our core equation of motion is f = d/dt (Iv) = Ia + vx* Iv
        # Imat is 6 by 6

        #tensordot not giving output with correct dimensions:
        # temp = np.tensordot(Imat[:, :, :], vec[:, :], axes=([1], [0]))

        temp = np.sum(Imat * vec[:, np.newaxis, :], axis=1)

        # Alternative for now: Iterate over each batch index
        # for i in range(batch_size):
        #     # Compute the dot product of A[:,:,i] and B[:,:,i]
        #     dot_product = np.dot(Imat[:, :, i], vec[:, i])
        #     # Accumulate the result
        #     temp[:,i] = dot_product

        temp = np.asarray(temp).reshape(-1)

        #TO CARLY: we don't need this code below because we already created an empty output array?
        # #create 1x6 vector of zeros

        vecXIvec = np.zeros((6, batch_size), dtype=np.float64)
        vec = np.asarray(vec)  # Convert vec to CuPy array

        vecXIvec[0] = -vec[2]*temp[1]   +  vec[1]*temp[2] + -vec[2+3]*temp[1+3] +  vec[1+3]*temp[2+3]
        vecXIvec[1] =  vec[2]*temp[0]   + -vec[0]*temp[2] +  vec[2+3]*temp[0+3] + -vec[0+3]*temp[2+3]
        vecXIvec[2] = -vec[1]*temp[0]   +  vec[0]*temp[1] + -vec[1+3]*temp[0+3] + vec[0+3]*temp[1+3]
        vecXIvec[3] = -vec[2]*temp[1+3] +  vec[1]*temp[2+3]
        vecXIvec[4] =  vec[2]*temp[0+3] + -vec[0]*temp[2+3]
        vecXIvec[5] = -vec[1]*temp[0+3] +  vec[0]*temp[1+3]
        res = vecXIvec


        #CATHERINE CODE
        # res[0] = -vec[2]*temp[1]   +  vec[1]*temp[2] + -vec[2+3]*temp[1+3] +  vec[1+3]*temp[2+3]
        # res[1] =  vec[2]*temp[0]   + -vec[0]*temp[2] +  vec[2+3]*temp[0+3] + -vec[0+3]*temp[2+3]
        # res[2] = -vec[1]*temp[0]   +  vec[0]*temp[1] + -vec[1+3]*temp[0+3] + vec[0+3]*temp[1+3]
        # res[3] = -vec[2]*temp[1+3] +  vec[1]*temp[2+3]
        # res[4] =  vec[2]*temp[0+3] + -vec[0]*temp[2+3]
        # res[5] = -vec[1]*temp[0+3] +  vec[0]*temp[1+3]

def vxIv_cupy(vec, Imat, res, batch_size):
        # necessary component in differentiating Iv (product rule).
        # We express I_dot x v as v x (Iv) (see Featherstone 2.14)
        # our core equation of motion is f = d/dt (Iv) = Ia + vx* Iv
        # Imat is 6 by 6

        #tensordot not working: we want (6, batch_size), getting (6, batch_size, batch_size)
        # temp = cp.tensordot(Imat[:, :, :], vec[:, :], axes=([1], [0]))

        temp = cp.sum(Imat * vec[:, cp.newaxis, :], axis=1)

        # Alterantive for now: Iterate over each batch index
        # for i in range(batch_size):
        #     # Compute the dot product of A[:,:,i] and B[:,:,i]
        #     dot_product = cp.dot(Imat[:, :, i], vec[:, i])
        #     # Accumulate the result
        #     temp[:,i] = dot_product

        temp = cp.asarray(temp).reshape(-1)

        #TO CARLY: same as above?
        #create 1x6 vector of zeros
        vecXIvec = cp.zeros((6, batch_size), dtype=cp.float64)
        vec = cp.asarray(vec)  # Convert vec to CuPy array

        vecXIvec[0] = -vec[2]*temp[1]   +  vec[1]*temp[2] + -vec[2+3]*temp[1+3] +  vec[1+3]*temp[2+3]
        vecXIvec[1] =  vec[2]*temp[0]   + -vec[0]*temp[2] +  vec[2+3]*temp[0+3] + -vec[0+3]*temp[2+3]
        vecXIvec[2] = -vec[1]*temp[0]   +  vec[0]*temp[1] + -vec[1+3]*temp[0+3] + vec[0+3]*temp[1+3]
        vecXIvec[3] = -vec[2]*temp[1+3] +  vec[1]*temp[2+3]
        vecXIvec[4] =  vec[2]*temp[0+3] + -vec[0]*temp[2+3]
        vecXIvec[5] = -vec[1]*temp[0+3] +  vec[0]*temp[1+3]
        res = vecXIvec
        #return vecXIvec #return a 1d

        #catherine code
        res[0] = -vec[2]*temp[1]   +  vec[1]*temp[2] + -vec[2+3]*temp[1+3] +  vec[1+3]*temp[2+3]
        res[1] =  vec[2]*temp[0]   + -vec[0]*temp[2] +  vec[2+3]*temp[0+3] + -vec[0+3]*temp[2+3]
        res[2] = -vec[1]*temp[0]   +  vec[0]*temp[1] + -vec[1+3]*temp[0+3] + vec[0+3]*temp[1+3]
        res[3] = -vec[2]*temp[1+3] +  vec[1]*temp[2+3]
        res[4] =  vec[2]*temp[0+3] + -vec[0]*temp[2+3]
        res[5] = -vec[1]*temp[0+3] +  vec[0]*temp[1+3]

**vxIv Batch Testing, Comparing to CPU and GPU with CuPy**

In [22]:
# COMPARING to batched
batch_size = 100
# vec is a 6 by 1 matrix
h_vec_batched = np.ones((6, batch_size),  dtype=np.float64)
d_vec_batched = cp.asarray(h_vec_batched)

#S should be a 6 by 6 matrix
h_I_batched = np.ones((6, 6, batch_size),  dtype=np.float64)
d_I_batched = cp.asarray(h_I_batched)

#vec output is a 6 by 6 matrix
h_output_batched = np.zeros((6, batch_size), dtype=np.float64)
d_output_batched = cp.zeros((6, batch_size), dtype=cp.float64)

#mxS output is a 6 by 6 matrix
h_mxS_output_batched = np.zeros((6, batch_size), dtype=np.float64)
d_mxS_output_batched = cp.zeros((6, batch_size), dtype=cp.float64)

alpha = 0.1

#CPU/with numpy
vxIv_numpy(h_vec_batched, h_I_batched, h_output_batched, batch_size) #warm-up once
print("vxIV shape: ", h_output_batched.shape)
#testing in loop of 100
startnext = timer()
for i in range(100):
 vxIv_numpy(h_vec_batched, h_I_batched, h_output_batched, batch_size)
print("CPU without jit Batched vxIv: " + str(timer() - startnext))

#GPU/with cupy
vxIv_cupy(d_vec_batched, d_I_batched, d_output_batched, batch_size) #warm-up once
print("vxIV shape: ", d_output_batched.shape)
#testing in loop of 100
startnext = timer()
for i in range(100):
  vxIv_cupy(d_vec_batched, d_I_batched, d_output_batched, batch_size)
cp.cuda.Stream.null.synchronize()
print("CuPy Batched vxIv: " + str((timer() - startnext)))

vxIV shape:  (6, 100)
CPU without jit Batched vxIv: 0.009317840999983673
vxIV shape:  (6, 100)
CuPy Batched vxIv: 0.17895130699997708


# RNEA: putting everything together

## Forward Pass

### numpy

In [41]:
def rnea_fpass_numpy(num_joints, parent_id_arr, xmat_func_arr, S_arr, Imat_arr, crOp_output, mxS_output, vxIv_output, batch_size, q, qd, qdd = None, GRAVITY = -9.81):
        """
        Forward Pass for RNEA algorithm. Computes the velocity and acceleration of each body in the tree necessary to produce a certain trajectory

        OUTPUT:
        v : input qd is specifying value within configuration space with assumption of one degree of freedom.
        Output velocity is in general body coordinates and specifies motion in full 6 degrees of freedom
        """
        # assert len(q) == len(qd), "Invalid Trajectories"
        # not sure should equal num links or num joints.

        '''1: Getting robot num joints '''
        ''' IIWA: number of joints? '''
        #assert len(q) == self.robot.get_num_joints(), "Invalid Trajectory, must specify coordinate for every body"
        # assert len(q) == num_joints

        # allocate memory
        # n = len(q)
        n = num_joints

        batch_size = 10000
        v = np.zeros((6,n, batch_size))
        a = np.zeros((6,n, batch_size))
        f = np.zeros((6,n, batch_size))

        gravity_vec = np.zeros((6, batch_size)) # model gravity as a fictitious base acceleration.
        # all forces subsequently offset by gravity.
        gravity_vec[5] = -GRAVITY # a_base is gravity vec, linear in z direction

        # forward pass
        # vi = vparent + Si * qd_i
        # differentiate for ai = aparent + Si * qddi + Sdi * qdi

        for ind in range(n):
            """ 2. Get parent ID"""
            # parent_ind = self.robot.get_parent_id(ind)
            parent_ind = parent_id_arr[ind]
            """ 3. Get XMat Func by ID"""
            """ 4. Get S by ID by ID"""
            # Xmat = robot.get_Xmat_Func_by_id(ind)(q[ind]) # the coordinate transform that brings into base reference frame
            # Xmat = xmat_func_arr[ind](q[ind])
            Xmat = xmat_func_arr[ind, :, :, :]
            # S = self.robot.get_S_by_id(ind) # Subspace matrix
            S = S_arr[ind]
            # compute v and a
            if parent_ind == -1: # parent is base
                # v_base is zero so v[:,ind] remains 0
                # Batched, inefficient code: a[:, ind, :] = np.tensordot(Xmat[:, :, :], gravity_vec[:, :], axes=([1], [0]))[:, :, 0]
                #Temp solution: Dimensions of a[:, ind] should be (6, 10)
                # for i in range(batch_size):
                #     # Compute the dot product of A[:,:,i] and B[:,:,i]
                #     mat_mul = Xmat[:, :, i] @ gravity_vec[:, i]
                #     a[:,ind, i] = mat_mul

                a[:, ind, :] = np.sum(Xmat*gravity_vec[:, np.newaxis, :], axis=1)
            else:
                # Batched but inefficient code: v[:, ind, :] = np.tensordot(Xmat[:, :, :], v[:, ind, :], axes=([1], [0]))
                # for i in range(batch_size):
                #     # Compute the dot product of A[:,:,i] and B[:,:,i]
                #     mat_mul = Xmat[:, :, i] @ v[:, parent_ind, i]
                #     v[:,ind, i] = mat_mul

                v[:, ind, :] = np.sum(Xmat*v[:, ind, :], axis=1)

                # batched but inefficient: a[:, ind, :] = np.tensordot(Xmat[:, :, :], a[:, ind, :], axes=([1], [0]))
                # for i in range(batch_size):
                #     # Compute the dot product of A[:,:,i] and B[:,:,i]
                #     mat_mul = Xmat[:, :, i] @ a[:, parent_ind, i]
                #     a[:,ind, i] = mat_mul

                a[:, ind, :] = np.sum(Xmat*a[:, ind, :], axis=1)

            v[:,ind, :] += S*qd[ind] # S turns config space into actual velocity
            """ 5. self.mxS """
            # a[:,ind] += self.mxS(S,v[:,ind],qd[ind])
            mxS_numpy(S,v[:,ind, :], crOp_output, mxS_output, qd[ind])
            a[:, ind, :] += mxS_output
            if qdd is not None:
                a[:,ind, :] += S*qdd[ind]

            # compute f
            """ 5. robot.get_Imat_by_id """
            """ 6. self.vxIv """
            # Imat = self.robot.get_Imat_by_id(ind)
            Imat = Imat_arr[ind, :, :]
            # f[:,ind] = Imat@a[:,ind] + vxIv_numpy(v[:,ind],Imat)
            # for i in range(batch_size):
            #     # Compute Imat@a[:,ind]
            #     #mat_mul = Xmat[:, :, i] @ gravity_vec[:, i]
            #     mat_mul = Xmat[:, :, i] @ a[:, ind, i]
            #     f[:,ind, i] = mat_mul
            temp = np.sum(Imat*a[:, ind, :], axis=1)
            vxIv_numpy(v[:,ind, :],Imat, vxIv_output, batch_size)
            f[:, ind, :] = temp + vxIv_output

        return (v,a,f)

### Testing Forward Pass

In [46]:
batch_size = 10000

#xmat_func_arr has shape (7, 6, 6)
print("xmat_func_arr shape:", xmat_func_arr.shape)
h_xmat_func_arr_batched = np.repeat(xmat_func_arr[:, :, :, np.newaxis], batch_size, axis=3)
d_xmat_func_arr_batched = cp.asarray(h_xmat_func_arr_batched)
print("h_xmat_func_arr_batched shape:", h_xmat_func_arr_batched.shape)

#S_arr has shape (7, 6)
print("S_arr shape:", S_arr.shape)
h_S_arr_batched = np.repeat(S_arr[:, :, np.newaxis], batch_size, axis=2)
d_S_arr_batched = cp.asarray(h_S_arr_batched)
print("h_S_arr_batched shape:", h_S_arr_batched.shape)

#Imat_arr has shape (7, 6, 6)
print("Imat_arr shape:", Imat_arr.shape)
h_Imat_arr_batched = np.repeat(Imat_arr[:, :, :, np.newaxis], batch_size, axis=3)
d_Imat_arr_batched = cp.asarray(h_Imat_arr_batched)
print("h_Imat_arr_batched shape:", h_Imat_arr_batched.shape)

#q has shape (7,)
print("q shape:", q.shape)
h_q_batched = np.repeat(q[:, np.newaxis], batch_size, axis=1)
d_q_batched = cp.asarray(h_q_batched)
print("h_q_batched shape:", h_q_batched.shape)

#qd has shape (7,)
print("qd shape:", qd.shape)
h_qd_batched = np.repeat(qd[:, np.newaxis], batch_size, axis=1)
d_qd_batched = cp.asarray(h_qd_batched)
print("h_qd_batched shape:", h_qd_batched.shape)

#cross_op output
h_crOp_output_batched = np.zeros((6, 6, batch_size), dtype=np.float64)
d_crOp_output_batched = cp.zeros((6, 6, batch_size), dtype=cp.float64)

#mxs output
h_mxS_output_batched = np.zeros((6, batch_size), dtype=np.float64)
d_mxS_output_batched = cp.zeros((6, batch_size), dtype=cp.float64)

#vxiv output
h_vxIv_output_batched = np.zeros((6, batch_size), dtype=np.float64)
d_vxIv_output_batched = cp.zeros((6, batch_size), dtype=cp.float64)

# print("v", v)
# print("a", a)
# print("f", f)



xmat_func_arr shape: (7, 6, 6)
h_xmat_func_arr_batched shape: (7, 6, 6, 10000)
S_arr shape: (7, 6)
h_S_arr_batched shape: (7, 6, 10000)
Imat_arr shape: (7, 6, 6)
h_Imat_arr_batched shape: (7, 6, 6, 10000)
q shape: (7,)
h_q_batched shape: (7, 10000)
qd shape: (7,)
h_qd_batched shape: (7, 10000)


In [43]:
itr = 100
# For reference, batch size is 10000 above
#warm-up once

#h_mxS_output_batched = np.zeros((6, batch_size), dtype=np.float64)
#d_mxS_output_batched = cp.zeros((6, batch_size), dtype=cp.float64)

# function header: rnea_fpass_numpy(num_joints, parent_id_arr, xmat_func_arr, S_arr,
#                  Imat_arr, crOp_output,
#                  mxS_output, vxIv_output, batch_size, q, qd, qdd = None, GRAVITY = -9.81)

v,a,f = rnea_fpass_numpy(n, parent_id_arr, h_xmat_func_arr_batched, h_S_arr_batched,
                         h_Imat_arr_batched, h_crOp_output_batched,
                         h_mxS_output_batched, h_vxIv_output_batched, batch_size,
                         h_q_batched, h_qd_batched, qdd = None, GRAVITY = -9.81)

startnext = timer()
for i in range(itr):
  rnea_fpass_numpy(n, parent_id_arr, h_xmat_func_arr_batched, h_S_arr_batched,
                         h_Imat_arr_batched, h_crOp_output_batched,
                         h_mxS_output_batched, h_vxIv_output_batched, batch_size,
                         h_q_batched, h_qd_batched, qdd = None, GRAVITY = -9.81)
print("GPU batched fpass with numpy: " + str((timer() - startnext)))

GPU batched fpass with numpy: 3.0550494880000088


### Numpy



In [29]:
def rnea_bpass_numpy(S_arr, parent_id_arr, xmat_func_arr, q, qd, f, USE_VELOCITY_DAMPING = False):
        # allocate memory
        n = len(q) # assuming len(q) = len(qd)
        c = np.zeros((n, batch_size))

        # backward pass
        # seek to calculate force transmitted from body i across joint i (fi) from the outside in.
        # fi = fi^B (net force) - fi^x (external forces, assumed to be known) - sum{f^j (all forces from children)}.
        # Start with outermost as set of children is empty and go backwards to base.
        for ind in range(n-1,-1,-1):
            S = S_arr[ind]
            # compute c
            # c[ind, :] = np.tensordot(np.transpose(S),f[:,ind, :], axes=([1], [0]))[:, 0]
            # for i in range(batch_size):
            #   mat_mul = np.transpose(S[:, i]) @ f[:, ind, i]
            #   c[ind, i] = mat_mul

            c[ind, :] = np.sum(S*f[:, ind, :], axis=0)

            # update f if applicable
            parent_ind = parent_id_arr[ind]
            if parent_ind != -1:
                Xmat = xmat_func_arr[ind, :, :]
                # temp = np.tensordot(np.transpose(Xmat), f[:,ind, :], axes=([1], [0]))[0, :, :]
                # for i in range(batch_size):
                #   mat_mul = np.transpose(Xmat[:, :, i]) @ f[:, ind, i]
                #   temp[:, i] = mat_mul
                temp = np.sum(Xmat*f[:, ind, np.newaxis, :], axis=1)
                f[:,parent_ind, :] = f[:,parent_ind, :] + temp

        # add velocity damping (defaults to 0)
        # if USE_VELOCITY_DAMPING:
        #     for k in range(n):
        #         c[k] += self.robot.get_damping_by_id(k) * qd[k]

        return (c,f)

### Testing Backward Pass

In [54]:
itr = 100
#for reference, batch_size is 10000 above

#first run Numpy fpass
v,a,f = rnea_fpass_numpy(n, parent_id_arr, h_xmat_func_arr_batched, h_S_arr_batched,
                         h_Imat_arr_batched, h_crOp_output_batched,
                         h_mxS_output_batched, h_vxIv_output_batched, batch_size,
                         h_q_batched, h_qd_batched, qdd = None, GRAVITY = -9.81)

#wardm-up once
c, f = rnea_bpass_numpy(h_S_arr_batched, parent_id_arr, h_xmat_func_arr_batched, h_q_batched, h_qd_batched, f, USE_VELOCITY_DAMPING = False)

startnext = timer()
for i in range(itr):
  rnea_bpass_numpy(h_S_arr_batched, parent_id_arr, h_xmat_func_arr_batched, h_q_batched, h_qd_batched, f, USE_VELOCITY_DAMPING = False)
print("GPU Batched bpass with numpy: " + str((timer() - startnext)))



GPU Batched bpass with numpy: 0.5499349629999415


## Full RNEA (numpy)

In [57]:
def rnea_numpy(q, qd, qdd = None, GRAVITY = -9.81, USE_VELOCITY_DAMPING = False):
      #print("test1")

      # forward pass
      v,a,f = rnea_fpass_numpy(n, parent_id_arr, h_xmat_func_arr_batched, h_S_arr_batched,
                         h_Imat_arr_batched, h_crOp_output_batched,
                         h_mxS_output_batched, h_vxIv_output_batched, batch_size,
                         h_q_batched, h_qd_batched, qdd = None, GRAVITY = -9.81)

      #print("test2")

      # backward pass
      (c,f) = rnea_bpass_numpy(h_S_arr_batched, parent_id_arr,h_xmat_func_arr_batched,
                               h_q_batched, h_qd_batched,
                               f, USE_VELOCITY_DAMPING = False)
      return (c,v,a,f)

# Testing Full RNEA: Numpy

In [58]:
#startnext = timer()
for i in range(itr):
    print(rnea_numpy(q, qd, qdd = None, GRAVITY = -9.81, USE_VELOCITY_DAMPING = False))
#print("CPU Batched RNEA: " + str((timer() - startnext)))

Streaming output truncated to the last 5000 lines.

       [[0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
        ...,
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ]],

       [[0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
        ...,
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ]],

       [[9.81, 9.81, 9.81, ..., 9.81, 9.81, 9.81],
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
        ...,
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , ..., 0.  , 